In [13]:
import numpy as np
import cv2

OUTPUT_VIDEO_FILE_PATH = "../data/input/test_video.mp4"

cap = cv2.VideoCapture(OUTPUT_VIDEO_FILE_PATH)

# take first frame of the video
ret, frame = cap.read()

# setup initial location of window
x, y, w, h = 793, 421, 198, 133  # simply hardcoded the values
track_window = (x, y, w, h)

# set up the ROI for tracking
roi = frame[y:y+h, x:x+w]
hsv_roi = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)
mask = cv2.inRange(hsv_roi, np.array((0., 0., 50.)), np.array((255., 255., 255.)))
roi_hist = cv2.calcHist([hsv_roi], [0], mask, [180], [0, 180])
cv2.normalize(roi_hist, roi_hist, 0, 255, cv2.NORM_MINMAX)

# Setup the termination criteria, either 10 iteration or move by at least 1 pt
term_crit = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 1)
while True:
    ret, frame = cap.read()
    if ret == True:
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        dst = cv2.calcBackProject([hsv], [0], roi_hist, [0, 180], 1)

        # apply camshift to get the new location
        ret, track_window = cv2.CamShift(dst, track_window, term_crit)

        # draw it on image
        pts = cv2.boxPoints(ret)
        pts = np.intp(pts)
        img2 = cv2.polylines(frame, [pts], True, 255, 2)
        cv2.imshow('img2', img2)
        k = cv2.waitKey(1000)
        if k == ord('q'):
            break
    else:
        break

cv2.destroyAllWindows()